In [1]:
!pip install soynlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 4.1 MB/s eta 0:00:00


In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=dbe0dfd5996108fcb69ba10efc1f61d93cb602aac537161cbbda84c11915144f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
import pandas as pd
import numpy as np
import re
from soynlp.normalizer import emoticon_normalize
from soynlp.tokenizer import RegexTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from langdetect import detect
# 데이터 불러오기
data = pd.read_excel("/content/토리든 토너 891개.xlsx")

reviews = data['필드3']

tokenizer = RegexTokenizer()

stopwords_url = "https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt"
stopwords = pd.read_csv(stopwords_url, header=None)[0].tolist()

def remove_special_characters(text):
    text = emoticon_normalize(text)
    # 모든 특수문자 및 이모티콘 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 특정 기호 수정 (예: ㅋ, ㅎ, ㅜ, ㅠ 등)
    text = re.sub(r'(ㅋ|ㅎ|ㅜ|ㅠ)+', '', text)
    text = re.sub(r'\s+', ' ', text)    # 공백 제거
    text = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', text)    # 한글 자음, 모음 제거
    text = re.sub('[^\w\s\n]', '', text)
    return text
def remove_non_korean_reviews(texts):
    filtered_texts = []
    for text in texts:
        try:

            if detect(text) != 'ko':
                continue
        except:
            # 언어 감지에 실패한 경우도 제외
            continue
        filtered_texts.append(text)
    return filtered_texts

# 전처리 함수 정의
def preprocess_text(text):
    text = remove_special_characters(text)
    tokens = tokenizer.tokenize(text)
    tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

# 전처리된 리뷰 데이터 생성
preprocessed_reviews = [preprocess_text(review) for review in reviews]

# 데이터프레임으로 변환
preprocessed_data = pd.DataFrame({'preprocessed_review': preprocessed_reviews})

# 전처리된 데이터를 엑셀 파일로 저장
preprocessed_data_file = "preprocessed_toner_reviews.xlsx"
preprocessed_data.to_excel(preprocessed_data_file, index=False)

/usr/local/lib/python3.10/dist-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))
